In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import time
import json

In [3]:
DRIVER_PATH = "../driver/chromedriver"

### Explore College Search

In [ ]:
# options = webdriver.ChromeOptions()
# options.add_argument('--disable-blink-features=AutomationControlled')
# options.add_argument("--disable-extensions")
# options.add_experimental_option('useAutomationExtension', False)
# options.add_experimental_option("excludeSwitches", ["enable-automation"])

college_name_elements = {
    "colleges": []
}

index = 0

try:
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(executable_path=DRIVER_PATH) # options=options
    #driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    driver.get("https://www.collegedata.com/college-search")

    time.sleep(10)

    body = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )

    last_height = driver.execute_script("return document.body.scrollHeight")
    print(last_height)

    while True:
        
        time.sleep(10)
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(10)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        time.sleep(10)

        print(new_height)

        if new_height == last_height:
            college_links_by_names = driver.find_elements(by=By.CLASS_NAME, value="Link_link__a-VS4")
            time.sleep(60)
            for element in college_links_by_names:
                college_name = element.find_element_by_css_selector("h3")
                college_name_elements["colleges"].append({
                    "college_name": college_name.text,
                    "link": element.get_attribute("href")
                })
            time.sleep(120)
            break
        last_height = new_height
finally:
    driver.quit()
    with open("./college_links.json", "w") as f:
        f.write(json.dumps(college_name_elements, indent=4))


### Explore Single College Info

In [6]:
driver = webdriver.Chrome(executable_path=DRIVER_PATH)

collegedata_selection_of_student = {}

importance = ["Very Important", "Important", "Considered", "Not Considered"]

with open("./college_links.json") as f:
    college_links = json.load(f)

for each_college_info in college_links["colleges"]:
    cur_college_data = {}
    cur_college_name = each_college_info["college_name"]
    driver.get(each_college_info["link"] + "/admission")

    time.sleep(5)
    
    sections = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".CollegeProfileContent_section__kPeSs"))
    )
    
    # driver.find_elements_by_css_selector(".CollegeProfileContent_section__kPeSs")

    selection_of_students_table = sections[2].find_element_by_css_selector(".CollegeProfileContent_content__1hJCl > .Table_unlined__29Tmx > tbody")

    metrics = selection_of_students_table.find_elements_by_css_selector("tr")

    for metric in metrics:
        elements = metric.find_elements_by_css_selector("td")
        index = 0
        metric_name = ""
        for element in elements:
            if index == 0:
                metric_name = element.text
            else:
                if element.text == "X":
                    cur_college_data[metric_name] = importance[index - 1]
                    break
            index += 1

    driver.quit()
    collegedata_selection_of_student[cur_college_name] = cur_college_data

with open("./collegedata_selection_of_student.json", "w") as f:
    f.write(json.dumps(collegedata_selection_of_student, indent=4))

/Users/minzhou/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.
/Users/minzhou/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
/Users/minzhou/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
/Users/minzhou/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead


MaxRetryError: HTTPConnectionPool(host='localhost', port=52131): Max retries exceeded with url: /session/bc252e5f284b2f3570c6176678b0911e/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ffd9a3d5e10>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [49]:
collegedata_selection_of_student

{'Essay': 'Very Important',
 'Interview': 'Very Important',
 "Level of Applicant's Interest": 'Very Important',
 'Particular Talent/Ability': 'Very Important'}